# <center> Практическое задание по теме "Циклы"

<img src=https://1gai.ru/uploads/posts/2020-12/1608804716_54441.png width=300 align="right">

→ Поздравляем с освоением важных для анализа данных конструкций Python (переменные, структуры данных, условные операторы и циклы)!

Настало время промежуточной практики, чтобы закрепить все полученные в предыдущих модулях навыки! Мы будем применять их на реальном проекте. Сегодня мы обратимся к классике: займемся анализом текстов на примере «Войны и мира» Льва Николаевича Толстого!

В рамках практического кейса мы сначала вместе познакомимся с исходными данными и произведем некоторые манипуляции над ними. После чего вам предстоит самостоятельно выполнить несколько заданий на тему поиска наиболее значимых слов в тексте с помощью методов статистического анализа текста. 

> Итак, приступим!

## <center> Знакомимся с данными

Текст произведения мы взяли в библиотеке [lib.ru](http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0073.shtml) и провели первоначальную обработку. Поскольку наша цель — обработка слов из этого произведения, мы разбили текст на слова и вывели каждое слово в отдельной строке. Кроме того, в местах, где начинаются главы, мы вывели строку `"[new chapter]"`.

> Исходный текстовый файл хранится в общем доступе и находится [здесь](https://raw.githubusercontent.com/SkillfactoryDS/Datasets/master/war_peace_processed.txt).

Для начала скачаем текст книги по ссылке.

In [1]:
# Импортируем библиотеку для выполнения HTTP-запросов в интернет
import requests 

# Читаем текстовый файл по url-ссылке
data = requests.get("https://raw.githubusercontent.com/SkillfactoryDS/Datasets/master/war_peace_processed.txt").text

# Предобрабатываем текстовый файл
data = data.split('\n')
data.remove('')
data = data + ['[new chapter]']

# Выводим первые 100 слов из книги
print(data[:100])

['1', 'в', 'два', 'раза', 'короче', 'и', 'в', 'пять', 'раз', 'интереснее', '2', 'почти', 'нет', 'философических', 'отступлений', '3', 'в', 'сто', 'раз', 'легче', 'читать', 'весь', 'французский', 'текст', 'заменен', 'русским', 'в', 'переводе', 'самого', 'толстого', '4', 'гораздо', 'больше', 'мира', 'и', 'меньше', 'войны', '5', 'хеппи-энд', 'эти', 'слова', 'я', 'поместил', 'семь', 'лет', 'назад', 'на', 'обложку', 'предыдущего', 'издания', 'указав', 'в', 'аннотации', 'первая', 'полная', 'редакция', 'великого', 'романа', 'созданная', 'к', 'концу', '1866', 'года', 'до', 'того', 'как', 'толстой', 'переделал', 'его', 'в', '1867--1869', 'годах', '--', 'и', 'что', 'я', 'использовал', 'такие-то', 'публикации', 'думая', 'что', 'все', 'всё', 'знают', 'я', 'не', 'объяснил', 'откуда', 'взялась', 'эта', 'первая', 'редакция', 'я', 'оказался', 'неправ', 'и', 'в', 'результате', 'оголтелые', 'и']


## <center> Работаем с данными

Для начала найдем общее количество слов и количество уникальных слов в тексте

In [2]:
# Превращаем список в множество, удаляя дублирующиеся слова
word_set = set(data)
# Удаляем из множества слово, символизирующее раздел между главами
word_set.discard('[new chapter]')
# Выводим результаты
print('Общее количество слов: {}'.format(len(data)))
print('Общее количество уникальных слов: {}'.format(len(word_set)))

Общее количество слов: 300080
Общее количество уникальных слов: 38210


Давайте напишем программу, которая посчитает частоту каждого слова. Для этого создадим словарь, ключами которого будут являться слова, а значения - количество вхождений этого слова в текст произведения. Заодно подсчитаем количество глав

In [3]:
# Инициализируем пустой словарь
word_counts = {}
# Инициализируем количество глав
count_chapter = 0
# Создаем цикл по всем словам из списка слов
for word in data:
    # Проверяем, что текущее слово - обозначение новой главы
    if word == '[new chapter]':
        # Если условие выполняется, то увеличиваем количество глав на 1
        count_chapter += 1
        # Переходим на новую итерацию цикла
        continue
    # Проверяем, что текущего слова еще нет в словаре слов
    if word not in word_counts:
        # Если условие выполняется, инициализируем новый ключ 1
        word_counts[word] = 1
    else:
        # В противном случае, увеличиваем количество слов на 1
        word_counts[word] += 1

# Выводим количество глав
print('Количество глав: {}'.format(count_chapter))

# Создаем цикл по ключам и их порядковым номерам полученного словаря
for i, key in enumerate(word_counts):
    # Выводим только первые 10 слов
    if i == 10:
        break
    print(key, word_counts[key])

Количество глав: 171
1 10
в 6997
два 215
раза 35
короче 3
и 14592
пять 61
раз 296
интереснее 4
2 12


Разделим все слова на главы. Для этого создадим список, в котором будем хранить списки - слова из определенной главы.

In [4]:
# Инициализируем общий список, в котором будем хранить списки слов в каждой главе
chapter_data = []
# Инициализируем список слов, в котором будет хранить слова одной главы
chapter_words = []

# Создаем цикл по всем словам из списка
for word in data:
    # Проверяем, что текущее слово - обозначение новой главы
    if word == '[new chapter]':
        # Если условие выполняется, добавляем список со словами из главы в общий список
        chapter_data.append(chapter_words)
        # Обновляем (перезаписываем) список со словами из текущей главы
        chapter_words = []
    else:
        # В противном случае, добавляем текущее слово в список со словами из главы
        chapter_words.append(word)

# Проверяем, что у нас получилось столько же списков, сколько глав в произведении
print('Вложенный список содержит {} внутренних списка'.format(len(chapter_data)))
# Выведем первые 100 слов 0-ой главы
print(chapter_data[0][:100])

Вложенный список содержит 171 внутренних списка
['1', 'в', 'два', 'раза', 'короче', 'и', 'в', 'пять', 'раз', 'интереснее', '2', 'почти', 'нет', 'философических', 'отступлений', '3', 'в', 'сто', 'раз', 'легче', 'читать', 'весь', 'французский', 'текст', 'заменен', 'русским', 'в', 'переводе', 'самого', 'толстого', '4', 'гораздо', 'больше', 'мира', 'и', 'меньше', 'войны', '5', 'хеппи-энд', 'эти', 'слова', 'я', 'поместил', 'семь', 'лет', 'назад', 'на', 'обложку', 'предыдущего', 'издания', 'указав', 'в', 'аннотации', 'первая', 'полная', 'редакция', 'великого', 'романа', 'созданная', 'к', 'концу', '1866', 'года', 'до', 'того', 'как', 'толстой', 'переделал', 'его', 'в', '1867--1869', 'годах', '--', 'и', 'что', 'я', 'использовал', 'такие-то', 'публикации', 'думая', 'что', 'все', 'всё', 'знают', 'я', 'не', 'объяснил', 'откуда', 'взялась', 'эта', 'первая', 'редакция', 'я', 'оказался', 'неправ', 'и', 'в', 'результате', 'оголтелые', 'и']


In [5]:
chapter_data[15][100]

'в'

Подсчитаем, сколько раз каждое слово встречается в каждой из глав

In [6]:
# Инициализируем список, в котором будем хранить словари
chapter_words_count = []

# Создаем цикл по элементам внешнего списка со словами
for chapter_words in chapter_data:
    # Инициализируем пустой словарь, куда будем добавлять результаты
    temp = {}
    # Создаем цикл по элементам внутреннего списка
    for word in chapter_words:
        # Проверяем, что текущего слова еще нет в словаре
        if word not in temp:
            # Если условие выполняется, добавляем ключ в словарь
            temp[word] = 1
        else:
            # В противном случае, увеличиваем количество влождений слова в главу
            temp[word] += 1
    # Добавляем получившийся словарь в список
    chapter_words_count.append(temp)

# Выводим результат
print(chapter_words_count[0])

{'1': 1, 'в': 37, 'два': 3, 'раза': 1, 'короче': 1, 'и': 34, 'пять': 1, 'раз': 2, 'интереснее': 1, '2': 1, 'почти': 1, 'нет': 1, 'философических': 1, 'отступлений': 1, '3': 1, 'сто': 1, 'легче': 1, 'читать': 1, 'весь': 2, 'французский': 2, 'текст': 3, 'заменен': 1, 'русским': 1, 'переводе': 1, 'самого': 2, 'толстого': 5, '4': 1, 'гораздо': 1, 'больше': 1, 'мира': 3, 'меньше': 1, 'войны': 3, '5': 1, 'хеппи-энд': 1, 'эти': 2, 'слова': 2, 'я': 8, 'поместил': 1, 'семь': 1, 'лет': 2, 'назад': 1, 'на': 7, 'обложку': 1, 'предыдущего': 1, 'издания': 3, 'указав': 1, 'аннотации': 1, 'первая': 2, 'полная': 1, 'редакция': 2, 'великого': 1, 'романа': 5, 'созданная': 1, 'к': 4, 'концу': 2, '1866': 3, 'года': 5, 'до': 1, 'того': 1, 'как': 2, 'толстой': 6, 'переделал': 1, 'его': 5, '1867--1869': 1, 'годах': 2, '--': 10, 'что': 9, 'использовал': 2, 'такие-то': 1, 'публикации': 1, 'думая': 1, 'все': 2, 'всё': 7, 'знают': 1, 'не': 11, 'объяснил': 1, 'откуда': 1, 'взялась': 1, 'эта': 1, 'оказался': 1, 'не

In [7]:
chapter_words_count[15]['князю']

2

In [8]:
# Создаем цикл по ключам словаря - спискам слов и их порядковым номерам
for chapter_number, chapter_dict in enumerate(chapter_words_count):
    # Выводим только первые 5 глав
    if chapter_number == 5:
        break
    # Выводим номер главы
    print('-' * 40)
    print('Chapter: {}'.format(chapter_number))
    print('-' * 40)
    # Создаем цикл по ключам - словам и их порядковым номерам
    for j, word in enumerate(chapter_dict):
        # Выводим первые 10 слов из главы
        if j == 10:
            break
        print(word, chapter_dict[word])

----------------------------------------
Chapter: 0
----------------------------------------
1 1
в 37
два 3
раза 1
короче 1
и 34
пять 1
раз 2
интереснее 1
2 1
----------------------------------------
Chapter: 1
----------------------------------------
автора 1
я 20
пишу 2
до 1
сих 1
пор 1
только 5
о 2
князьях 1
графах 1
----------------------------------------
Chapter: 2
----------------------------------------
первая 1
----------------------------------------
Chapter: 3
----------------------------------------
-- 81
ну 5
что 44
князь 21
генуя 1
и 94
лукка 1
стали 1
не 57
больше 2
----------------------------------------
Chapter: 4
----------------------------------------
гостиная 1
анны 2
павловны 2
начала 1
понемногу 1
наполняться 1
приехала 3
высшая 1
знать 1
петербурга 2


Давайте резюмировать, что мы с вами уже получили:

* `word_set` - множество из всех слов, которые есть в книге

* `count_chapter` - количество глав в книге (171)

* `word_counts` - словарь, ключами которого являются слова, а значениями - количество вхождений этих слов в книгу

* `chapter_data` - список из 171 списка, где элементы вложенных списков - все слова из главы. Каждый список соответствует своей главе

* `chapter_words_count` - список из 171 словаря, где ключи - слова, а значения - количество слов в главе. Каждый словарь соответствует своей главе

Учтите, что эти данные могут пригодиться вам при выполнении дальнейших заданий.

> А теперь к заданиями!

## <center> Задания для самостоятельного решения

### Задание 1.

Давайте введем понятие частоты употребления отдельного слова в документе (`term frequency`, или `tf`). В нашем случае речь идёт не о документах, а о главах книги (выше мы писали, что в текстовом документе главы разделяются строкой '[new chapter]').

Формула для вычисления `term frequency` для слова `word`:
$$ tf_{word, chapter} = \frac {n_{word, chapter}} {n_{chapter}}$$

где 
* ${n_{word, chapter}}$ - сколько раз слово `word` встрачается в главе `chapter`, 
* $n_{chapter}$ - количество слов в главе `chapter`.


Например, слово `"гостья"` употребляется в 15-ой главе 10 раз (${n_{word, chapter}}$).(кстати, главы у нас нумеруются с 0). Общее количество слов в тексте 15-ой главы - 1359 ($n_{chapter}$). Тогда:

$$ tf_{гостья, 15} = \frac{10}{1359} \approx 0.007358$$

**Задание:** 

Напишите программу, которая позволит получать частоту употребления любого заданного слова `target_word` в заданной главе `target_chapter`. 

**Дополнительное требование:**

*Пострайтесь сделать программу максимально обобщенной. То есть желательно рассчитать характеристику `tf` для всех слов из каждой главы, чтобы впоследствии не было необходимости производить вычисления снова.*

**Подсказка:**

*Для этого вы можете для каждой главы создать словарь, ключами которого являются слова, а значения - частота употребления этого слова в этой главе*

**Протестируйте работу программы на нескольких словах и главах.**

In [9]:
# Имеем переменную chapter_words_count. Нам нужно каждое значение каждого словаря внутри этого списка разделить на количество слов в соответствующей главе.
tf_count = []                                               # Cписок словарей со значениями tf

for ch_num, word_dict in enumerate(chapter_words_count):
    tf_chapter = {}                                         # Словарь значений tf для данной главы. Наполняется во вложенном цикле.
    len_chapter = len(chapter_data[ch_num])                 # Посчитаем количество слов в главе ch_num
    
    for word, value in word_dict.items():                   # Итерируем словарь word_dict главы ch_num, добавляя в словарь tf_chapter ключи-слова и значения tf
        tf_chapter.setdefault(word, value/len_chapter)
    
    tf_count.append(tf_chapter)                             # Добавляем полученный словарь в список словарей

In [10]:
target_word = 'князь'
target_chapter = 12
try:
    print(tf_count[target_chapter][target_word])
except KeyError:
    print(f'Слова "{target_word}" нет в главе {target_chapter}')

0.012048192771084338


### Задание 2.

Пришло время познакомиться с понятием `document frequency`.

`Document frequency` (для удобства сократим до `df`) — это доля документов, в которых встречается искомое слово. 

Вычисляется по формуле:

$$ df_{word} = \frac{N_{word}}{N} $$, 

где 
* $N_{word}$ - число документов (глав) содержащих слово `word`, 
* $N$ - общее число документов (глав).

Объясним на примере: наш текст состоит из 171 главы ($N$), а слово `"человек"` встречается в 115 главах. Тогда:

$$ df_{человек} = \frac{115}{171} \approx 0.6725$$

**Задание:** 

Напишите программу, которая позволит вычислять document frequency для заданного слова `target_word` и выведить результат на экран.

**Дополнительное требование:**

*Пострайтесь сделать программу максимально обобщенной. То есть желательно рассчитать характеристику `df` для всех уникальных слов из книги, чтобы впоследствии не было необходимости производить вычисления снова.*

**Подсказка:**
*Для этого вы можете создать словарь, ключами которого являются слова из книги, а значения - доля документов, содержащих эти слова*

**Протестируйте работу программы на нескольких словах** 

In [11]:
# Можно было перевести списки во множества, чтобы не таскаться со значениями, которые нам все равно не нужны.
# Но т.к. большого выигрыша от этого я не увидел, то работаю со словарями

df_count = {}                                         # Словарь значений df, будем наполнять

for word in word_counts.keys():                       # итерируем уникалльные слова в тексте
    df_count.update({word: 0})                        # добавляем ключ-слово присваиваем значение 0
    for chapter_words in chapter_words_count:         # итеррируем словари по главам
        if chapter_words.get(word) is not None:       # Проверяем есть ли ключ (слово) в данной главе
            df_count[word] += 1                       # если есть увеличиваем занчение на 1
    df_count[word] = df_count[word]/count_chapter     # Пересчитываем результат в df, записываем в то же место.

In [12]:
target_word = 'человек'

try:
    print(df_count[target_word])
except KeyError:
    print(f'Слова "{target_word}" нет в тексте')

0.672514619883041


### Задание 3

Пришло время дать разъяснения: для чего мы делали вычисления выше и что нас ждет впереди?

> Если какое-то слово часто употребляется в документе, то, вероятно, этот документ что-то рассказывает о предмете/действии, описываемом этим словом. Скажем, если вы читаете книгу, в которой много раз употребляется слово `"заяц"`, то, вероятно, эта книга про зайцев.

> Однако, если вы возьмёте слово `"и"`, то оно будет встречаться почти в каждой книге много раз. 

Таким образом, если мы хотим найти наиболее значимые слова в книге, мы, с одной стороны, хотим найти наиболее частые слова, а с другой — убрать те, которые не несут важной информации, так как встречаются везде.

Такая задача хорошо решается с помощью `tf-idf` — статистической метрики для оценки важности слова в тексте. Другими словами, `tf-idf` — это «контрастность» слова в документе (насколько оно выделяется среди других слов). 

Формула для вычисления следующая:

`tf-idf = term frequency * inverse document frequency`

* `tf` — это частотность термина, которая измеряет, насколько часто термин встречается в документе.

* `idf` — это обратная документная частотность термина. Она измеряет непосредственно важность термина во всём множестве документов.

Чтобы получить `idf`, необходимо поделить 1 на полученную в Задании 2 документную частоту (`df`):

$$idf = \frac{1}{df}$$

Мы будем использовать не сырые значения `idf`, а их логарифмы, то есть $tf * log(idf)$. Сейчас мы не будем заострять внимания на том, почему следует использовать именно логарифм — это долгий разговор. Вернемся к нему, когда будем изучать методы машинного обучения для обработки текстов. Подробнее о `tf-idf` вы можете почитать [здесь](https://translated.turbopages.org/proxy_u/en-ru.ru.15518a02-63e76541-6895b80b-74722d776562/https/www.freecodecamp.org/news/how-to-process-textual-data-using-tf-idf-in-python-cd2bbc0a94a3/).

В качестве примера измерим `tf-idf` слова `"анна"` в главе 4. Слово `"анна"` встречается в указанной главе 7 раз, при этом в 4 главе 1060 слов, всего же слово `"анна"` упоминается в 32 главах из 171.

Таким образом, `tf-idf` данного слова в данной главе будет равно:

$$tf\_idf_{анна, 4} = tf * log(\frac{1}{df}) = \frac{7}{1060} * log(\frac {171}{32}) \approx 0.011067$$

**Примечание:** здесь используется натуральный логарифм по основанию $e$, однако в общем случае основание логарифма не имеет значения, так как характеристика `tf-idf` используется для сравнения контрастности слов между собой

**Задание**:

Напишите программу, которая позволяет вычислять значение `tf-idf` для заданного слова `target_word` в заданной главе `target_chapter`.

**Дополнительное требование:**

*Пострайтесь сделать программу максимально оптимальной. То есть желательно рассчитать характеристику `tf-idf` для всех слов из каждой главы книги, чтобы впоследствии не было необходимости производить вычисления снова.*

**Протестируйте работу программы на нескольких словах и главах.**

**Примечание:**

Натуральный логарифм можно вычислить с помощью функции [log](https://pythonim.ru/chisla/funktsiya-log-v-python) из встроенного в Python модуля math:


In [18]:
# Импортируем функцию log из модуля math:
from math import log
print(log(18))

2.8903717578961645


**Примечание.** 

**Модуль (библиотека) в Python** — это любой программный файл, который содержит в себе код, включая функции. В нашем случае math — это встроенный модуль, содержащий функционал для математических вычислений. Подробнее о math вы можете почитать [здесь](https://pythonworld.ru/moduli/modul-math.html). 

In [19]:
# Ваш код здесь
tf_idf_count = []                              # Создаем пустой список
for word_dict in tf_count:                     # Итерируем список по словарям
    tf_idf_dict = {}                           # Создаем пустой словарь, который будем заполнять значениями tf-idf
    for word, tf in word_dict.items():         # Словарь word_dict итерируем по элементам (слово, значение tf) 
        tf_idf = tf * log(1/df_count[word])    # Для каждого слова считаем tf_idf 
        tf_idf_dict.update({word: tf_idf})     # добавляем результат в словарь
    tf_idf_count.append(tf_idf_dict)           # добавляем получившийся словарь в список

Проверяем результат

In [20]:
target_word = 'анна'
target_chapter = 3
try:
    print(tf_idf_count[target_chapter][target_word])
except KeyError:
    print(f'Слова "{target_word}" нет в главе {target_chapter}')

0.009813990764927087


### Задание 4.

Теперь, когда мы умеем вычислять `tf-idf` для каждого слова в главе, мы можем найти те слова, которые являются самыми «контрастными» для данной главы, то есть они могут являться в своём роде заголовком для главы.

Например, для главы 3 наиболее значимыми словами будут:

`"анна", "павловна", "функе"`

**Задание:**

Напишите программу, которая позволяет вывести три слова, имеющие самое высокое значение `tf-idf` в заданной главе `target_chapter` в порядке убывания `tf-idf`.

In [21]:
# Ваш код здесь


In [22]:
# Это сложный способ
cont_words_number = 3                                # сколько слов ищем
target_chapter = 3
tf_idf_chapter = tf_idf_count[target_chapter].copy() # Т.к. мы будем удалять слова из словаря, то чтобы исходный оставался неизменным сделаем копию.
c_words_list = []                                    # найденные слова будем добавлять в список

for i in range(cont_words_number):                   # в каждой итерации цикла ищем самое котрастное слово и удаляем его из словаря
    max_tf_idf = float('-inf')                       # будем добавлять сюда максимальное значение котрастности 
    contrast_word = []                               # сюда добавим самое контрастное слово. Можно было сделать строкой, но строки неизменяемые - будет утечка памяти при перезаписи.
    for word, tf_idf in tf_idf_chapter.items():      # итерируем весь словарь главы по словам и значениям tf-idf
        if tf_idf > max_tf_idf:                      # проверка на "максимальность":
            max_tf_idf = tf_idf                      # если проверка прошла, то перезаписываем значение максимума...
            contrast_word = [word]                   # ...и заменяем промежуточное самое котрастное слово
    tf_idf_chapter.pop(contrast_word[0])             # после того как прошли весь словарь. удаляем это слово из словаря методом dict.pop()
    c_words_list.append(contrast_word[0])            # записываем найденое слово с список самых котрастных слов 

print(c_words_list)


['павловна', 'анна', 'функе']


In [25]:
# Этот гораздо проще, но в нем используются еще неизученные приемы
cont_words_number = 3
target_chapter = 3
tf_idf_chapter = tf_idf_count[target_chapter].copy()
sorted_dict = sorted(tf_idf_chapter.items(), key=lambda x:x[1], reverse=True)
for i, item in enumerate(sorted_dict):
    print(item[0])
    if i == cont_words_number - 1:
        break

павловна
анна
функе
